In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
L = 1
K = 200
C = 1000
rho = 3000
delta_x = 0.01
delta_t = 0.1
N_t = 10000
l = K/(C*rho)
a = l*delta_t/delta_x**2
x = np.linspace(-L/2, L/2, int(L/delta_x))
x = np.pad(x, (1, 1), 'constant', constant_values=0)
T_0 = [np.sin(np.pi*x_i/L) for x_i in x]

In [6]:
T = [T_0]

for t in tqdm(range(N_t)):
    T_new = np.zeros(int(L/delta_x)+2)
    T_old = T[-1]
    for i in range(1, T_new.shape[0]-1):
        T_new[i] = (1-2*a)*T_old[i]+a*(T_old[i-1]+T_old[i+1])
    T.append(T_new)

100%|██████████| 10000/10000 [00:00<00:00, 11494.82it/s]


In [18]:
import matplotlib; matplotlib.use("TkAgg")
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(x, T_0)
legend = plt.legend(title= "Time step")

def animate(i):
    line.set_ydata(T[i])
    #legend.set_text(i)
    return line,

# Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig=fig, func=animate, frames=N_t, init_func=init, interval=250, blit=True)

plt.xlabel("x")
plt.ylabel("T")
plt.title("Temperature development in the rod")
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
